# Week 48

In [2]:
try:
    import fysisk_biokemi
    print("Already installed")
except ImportError:
    %pip install -q "fysisk_biokemi[colab] @ git+https://github.com/au-mbg/fysisk-biokemi.git"

------------------------------------------------------------------------

## 1 Qualitative description of Michaelis-Menten kinetics

In [3]:
import fysisk_biokemi.widgets as fbw
from fysisk_biokemi.widgets.utils import enable_custom_widget_colab, disable_custom_widget_colab

import plotly
from IPython.display import display, HTML
enable_custom_widget_colab()
fbw.michaelis_menten_demo()

Use the widget created by the cell above to answer the following the
questions.

#### (a)

Setting $V_\mathrm{max} = 400$ and $[E]_\mathrm{tot} = 0.0005$, change
$K_M$ from 0.5 to 5 to 10 to 50. Describe the change in curve appearance
and explain it using the Michaelis-Menten (MM) equation.

#### (b)

What is the biological meaning of $K_M$?

#### (c)

Now set $[E]_\mathrm{tot} = 0.0005$, and $K_M = 5$, change
$V_\mathrm{max}$ from 40 to 100 to 900. Do you observe any changes?
Explain your answer using the widget and the MM equation

#### (d)

Now set $K_M = 5$ and $V_\mathrm{max} = 100$, change $[E]_\mathrm{tot}$
from 0.0005 to 0.05. Do you observe any change in curve appearance, in
$k_\mathrm{cat}$ or in $k_\mathrm{cat}/K_M$? Explain your answer

------------------------------------------------------------------------

## 2 The Michaelis Menten equation

Consider a reaction catalyzed by an enzyme obeying the Michaelis-Menten
kinetics model.

$$
E + S \underset{k_{-1}}{\stackrel{k_1}{\rightleftharpoons}} ES \stackrel{k_2}{\rightarrow} E + P
$$

$$
V_0 = V_{\mathrm{max}} \frac{[S]}{[S] + K_M}
$$

#### (a) Enzyme saturation

Explain what is meant by enzyme saturation.

#### (b)

Calculate the level of enzyme saturation, $f_{ES}$, at substrate
concentrations of

-   $[S] = 0.1 \cdot K_M$
-   $[S] = 0.5 \cdot K_M$
-   $[S] = 1 \cdot K_M$
-   $[S] = 10 \cdot K_M$
-   $[S] = 100 \cdot K_M$

In [4]:
... # Calculate here

------------------------------------------------------------------------

The Michaelis-Menten constant KM is defined as:

$$
K_M = \frac{k_{-1}+ k_2}{k_1}
$$

#### (c) Calulate $K_m$

In a specific reaction, the following rate constants were determined:

-   $k_1 = 7 \cdot 10^7 \cdot \mathrm{M}^{-1}\cdot \mathrm{s}^{-1}$,
-   $k^{-1} = 8 \cdot 10^5 \ \mathrm{s}^{-1}$
-   $k_2 = 10^3 \ \mathrm{s}^{-1}$

Calculate the value of $K_M$. Does $K_M$ approximate the dissociation
constant for the $ES$ complex in this case?

In [5]:
... # Calculate here

#### (d) Calculate $k_2$

Assume that a solution of an enzyme at a concentration of
$1 \cdot 10^{-7} \ \mathrm{M}$ with a substrate concentration of
$[S]=100\cdot K_M$ has
$V_0=10^{-4} \ \mathrm{M} \cdot \mathrm{min}^{-1}$

Given this information and with your answers to (b) and (c) in mind,
calculate the constant $k_2$. What is this constant also called?

In [6]:
... # Calculate here

#### (e) Lineweaver-Burk plot

The values of $V_\mathrm{max}$ and $K_M$ have historically been
determined from a Lineweaver-Burk plot.

How does the x- and y-intercepts in a Lineweaver-Burk plot relate to
$V_\mathrm{max}$ and $K_M$?

------------------------------------------------------------------------

## 3 Enzyme inhibitors (I)

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
pd.set_option('display.max_rows', 6)

An enzyme obeying the Michaelis-Menten kinetics model was tested for
substrate conversion in the absence and presence of an inhibitor, called
inhibitor1 at a concentration of
$[\textrm{I}] = 2.5 \cdot 10^{-3} \ \textrm{M}$. The data set is
contained in the file `enzyme-inhib-i.xlsx`. Using this data a
researcher wanted to determine the type of inhibition.

Start by loading the dataset

In [8]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [9]:
df = uploader.get_dataframe()
display(df)

#### (a) Convert units

Convert the concentrations of substrate and the initial velocities to
units given in M and $\mathrm{M}\cdot \mathrm{s}^{-1}$, respectively.

In [11]:
df['[S]_(M)'] = ...
df['V0_no_inhib_(M/s)'] = ...
... # This one you'll have to do all on your own.
display(df)

#### (b) Plot

Plot the initial velocities of both experiments as a function of
substrate concentration in one plot. Estimate $k_\mathrm{cat}$ and
$V_\mathrm{max}$ in the presence and absence of inhibitor from the plot.

In [13]:
fig, ax = plt.subplots(figsize=(6, 4))

... # Your code for plotting here
... # and here. 

ax.set_xlabel('[S] (M)')
ax.set_ylabel('$V_0$ (M/s)')
ax.legend()

------------------------------------------------------------------------

The researcher wanted to determine $V_\mathrm{max}$ and $K_M$ values for
both experiments in order to correctly conclude on the type of
inhibitor.

#### (c)

Determine $V_\mathrm{max}$ and $K_M$ by fitting.

Start writing the function to fit with

In [15]:
def ...
    ...

And use the `curve_fit`-function to find the parameters

In [17]:
popt, _ = ...
vmax, km = popt

# With inhibitor
popt, _ = ...
vmax_inhib, km_inhib = popt

# Printing
print('Without inhibitor:')
print(f'\t V_max = {vmax:.2e}')
print(f'\t Km = {km:.2e}')
print('With inhibitor:')
print(f'\t V_max = {vmax_inhib:.2e}')
print(f'\t Km = {km_inhib:.2e}')

#### (d)

What type of inhibitor is inhibitor1?

#### (e)

Calculate $K_i$.  

------------------------------------------------------------------------

## 4 Enzyme inhibitors (II)

In [19]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
pd.set_option('display.max_rows', 6)

Two enzyme inhibitors were identified as part of a drug discovery
program. To characterize the mechanism of action, the reaction kinetics
were analysed for the enzyme alone and in the presence of 5 $\mu$M of
each of the inhibitors as a function of substrate concentration
resulting in the data in the file `enzyme-inhib-ii.xlsx`

Load the dataset

In [20]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [21]:
df = uploader.get_dataframe()
display(df)

#### (a) Convert & plot

Convert the measurements to SI units and plot the reaction rate against
time for all three reactions.

Start by converting the units

In [23]:
... # Your code to convert units here.

And then plot

In [25]:
fig, ax = plt.subplots()

col_names = ['enz_(M/s)', 'inhibitor2_(M/s)', 'inhibitor3_(M/s)']

for col_name in col_names:
    ax.plot(..., ..., 'o', label=col_name) # Replace ... with your code here. 
    
ax.legend()
ax.set_xlabel('Substrate concentration [M]', fontsize=14)
ax.set_ylabel('$V_0$', fontsize=14)
plt.show()

Based on the appearance of these plots:

-   Can you determine the type of inhibition?
-   Alternatively, can you exclude some mechanisms?

#### (b) Intuit parameters

Based on your answer to in (a) what do you expect for the parameters
$K_M$ and $V_\mathrm{max}$?

-   Which dataset would have the highest $V_\mathrm{max}$, which the
    lowest?
-   What about $K_M$?

#### (c) Fit

As always, we need the function we’re fitting with

In [27]:
def michaelis_menten(S, Vmax, Km):
    return (Vmax * S) / (S + Km)

Given we’re fitting on multiple datasets we will make our lives a bit
easier by making a little function

In [28]:
def fit_parameters(df, col_name):
    popt, _ = curve_fit(..., ..., ...) # Replace ... with your code.
    vmax, km = popt
    # Printing
    print(f'{col_name}:')
    print(f'\t V_max = {vmax:.2e}')
    print(f'\t Km = {km:.2e}')

In [30]:
for col_name in col_names:
    fit_parameters(df, col_name)

#### (d) Change

In the fits above you will likely see that all the fitted parameters
change between the different data sets – some by a little and some by a
lot. How big does a change have to be to be considered significant?

#### (e) $K_i$

Determine the $K_i$ for each of the two inhibitors.

------------------------------------------------------------------------

## 5 A snake dependent enzyme

When (or *if* depending on your outlook in life) you are bit by a cobra,
the snake protein CVF interacts with the human protein FB to form a
proteolytically active enzyme in the blood called CVF:FB. CVF:FB enzyme
is able to cleave another human protein known as C5 into C5b and C5a
(C5a is the protein you visually inspected in your first PyMOL session).
With C5 as a substrate, CVF:FB obeys the Michaelis-Menten kinetics
model, whereas CVF alone does not show any catalytic activity towards
C5.

Now consider two different CVF proteins, CVF1 and CVF2 from two
different cobra species. Upon characterization it was found that CVF1:FB
has a $K_M = 0.036 \ \mu\text{M}$ and a
$k_{cat} = 0.43 \ \mathrm{min}^{-1}$, whereas CVF2:FB has a
$K_M = 14 \mu\mathrm{M}$ and a $k_{cat} = 0.73 \ \mathrm{min}^{-1}$. The
concentration of C5 in blood is $0.37 \ \mu\mathrm{M}$.

#### (a)

Convert the given $K_M$ values and the C5 concentration to SI-units
given in $\mathrm{M}$

In [32]:
# CVF1
K_m_cvf1 = ...
k_cat_cvf1 = ...

# CVF2
...
...

# Concentration
C_c5 = ...

#### (b)

Assuming that \[CVF1:FB\] $=$ \[CVF2:FB\]
$= 0.1\cdot 10^{-6} \ \mathrm{M}$, calculate $V_\mathrm{max}$ of both
enzymes.

In [34]:
V_max_cfv1 = ...
...

print(f"{V_max_cfv1 = :.3e}")
print(f"{V_max_cfv2 = :.3e}")

#### (c)

Calculate the fraction of enzyme:substrate complexes (CVFx:FB:C5) in
blood.

In [36]:
theta_cfv1 = ...
theta_cfv2 = ...

print(f"{theta_cfv1 = :.3f}")
print(f"{theta_cfv2 = :.3f}")

#### (d)

Calculate the initial velocities of C5 cleavage in blood by the two
enzymes.

In [38]:
v0_cfv1 = ...
v0_cfv2 = ...

print(f"{v0_cfv1 = :.3e}")
print(f"{v0_cfv2 = :.3e}")

#### (f)

Which of the two enzymes, CVF1:FB or CVF2:FB, cleaves the most C5
molecules in blood?

------------------------------------------------------------------------

## 6 Enzymatic behaviour of the enzyme ATCase

In [40]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd

The enzyme aspartate transcarbamoylase (ATCase) catalyzes the first
reaction in the biosynthesis of pyrimidines such as CTP as shown in the
reaction below:

![](https://raw.githubusercontent.com/au-mbg/fysisk-biokemi/refs/heads/main/course/lessons/figures/atcase.png)

ATCase does not obey the Michaelis-Menten kinetics model but instead
shows the behaviour recorded in the `enzyme-behav-atcase.csv` dataset.

The dataset consists of three columns; the aspartate concentration in mM
and the rate of formation of N-carbamyolaspartate with and without the
presence of CTP.

Load the dataset with the widget below

In [41]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [42]:
df = uploader.get_dataframe()
display(df)

#### (a) Plot the dataset

In [44]:
fig, ax = plt.subplots()

... # Put some code here
... # And here

ax.set_xlabel('[Aspartate] (mM)')
ax.set_ylabel('Rate of formation')
ax.set_yticklabels([]);

#### (b) Kinetic profile

Describe the kinetic profile of ATCase and explain what it tells us
about the way ATCase works. (You may find inspiration in the material
previously covered on protein-ligand interactions)

#### (c) Quarternary structure

What does the figure tell us about the quaternary structure of ATCase?

#### (d) DNA synthesis

Which other pyrimidine based trinucleotide besides CTP is used to
synthesize DNA?

#### (e) Effect of CTP

Qualitatively describe the effect of CTP on the rate of
N-carbamyolaspartate formation

#### (f) Physiological advantage

In fact many enzymes are regulated by certain end products in a fashion
similar to the CTP effect on ATCase. Can you explain why this might be a
physiological advantage?

#### Hill coefficient

Use regression to determine the Hill coefficient for each of the
datasets. Recall that the Hill equation is,

$$
\theta = \frac{[L]^n}{K_d + [L]^n}
$$

As usual start by writing a function implementing this equation

In [46]:
def hill_eq(L, Kd, n):
    return L**n / (Kd + L**n)

And then make the fit for each dataset

In [47]:
# We start by massaging the data into the right units etc.
L_M = df['aspartate'] * 10**(-3) # Convert to SI units in M.

# For the dataset with ctp we need to discard the elements that are NaN / not measured.
indices = np.argwhere(df['with_ctp'].notna()).flatten() # Don't worry too much about how this works - it just finds the indices of the non empty elements. 
L_M_ctp = L_M[indices]
theta_ctp = df['with_ctp'][indices]

# Then we're ready to fit
pcov, _ = ...
pcov_ctp, _ = ...
Kd, n = pcov
Kd_ctp, n_ctp = pcov_ctp

# Print
print('Without CTP')
print(f'   K_d = {Kd:.3e}')
print(f'   n = {n:.3f}')
print('With CTP')
print(f'   K_d = {Kd_ctp:.3e}')
print(f'   n = {n_ctp:.3f}')

Then we can plot

In [49]:
fig, ax = plt.subplots(figsize=(7, 4))

L_smooth = np.linspace(0, L_M.max())
theta_fit = hill_eq(L_smooth, Kd, n)
theta_ctp_fit = hill_eq(L_smooth, Kd_ctp, n_ctp)

ax.plot(L_M, df['without_ctp'], 'o', label='No CTP')
ax.plot(L_smooth, theta_fit, 'k')

ax.plot(L_M_ctp, theta_ctp, 'o', label='With CTP')
ax.plot(L_smooth, theta_ctp_fit, 'k')

ax.set_xlabel('[Aspartate] (M)')
ax.set_ylabel('Rate of formation')